<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Загрузим необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

Создадим наш датафрейм из загруженного файла

In [2]:
try:
    df = pd.read_csv ('insurance.csv')
except:
    df = pd.read_csv ('/datasets/insurance.csv')

Создадим функцию для просмотра данных в датасете

In [3]:
def first_look (df: pd.DataFrame, limit_na_category: int = 10) -> None:
    '''Функция получения первичной информации о датафрейме'''
    print ('------------- Первые 5 строк ------------')
    display(df.head())
    print('')
    print('')
    print ('------------- Типы данных ------------')
    print (df.info())
    print('')
    print('')
    print ('------------- Пропуски ------------')
    count = 0
    shape_0 = df.shape[0]
    for element in df.columns:
        if df[element].isna().sum() > 0:
            print(element, ' - ', df[element].isna().sum(), 'пропусков, ', round(df[element].isna().sum() * 100 / shape_0,2), '% от числа строк.' )
            count = +1
    if count == 0:
        print('Пропусков НЕТ')
        print('')
        print('')
    print ('------------- Дубликаты ------------')
    if df.duplicated().sum() > 0:
        print('Дубликатов: ', df.duplicated().sum())
    else:
        print('Дубликатов НЕТ')
    # ищем, есть ли категориальные столбцы (содержащие до limit_na_category уникальных значений) в датафрейме
    min_unique = limit_na_category + 1
    for element in df.columns:
        min_unique = min(min_unique, df[element].nunique())
        if min_unique <= limit_na_category:
            print('')
            print('')
    print('------------- Категориальные признаки ------------')
    for element in df.columns:
        if df[element].nunique() <= limit_na_category:
            print(element, ': ', df[element].nunique(), 'категории - ', df[element].unique(), )

Быстрый взгляд на наши данные

In [4]:
first_look (df)

------------- Первые 5 строк ------------


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0




------------- Типы данных ------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None


------------- Пропуски ------------
Пропусков НЕТ


------------- Дубликаты ------------
Дубликатов:  153










------------- Категориальные признаки ------------
Пол :  2 категории -  [1 0]
Члены семьи :  7 категории -  [1 0 2 4 3 5 6]
Страховые выплаты :  6 категории -  [0 1 2 3 5 4]


Наши данные состоят из 5 колонок (не считая колонки индекса) и 5000 строк (не считая строки с наименованием столбцов).

Типы данных в столбцах нас устраивают.

Пропусков нет.

Имеется 153 дубликата данных и 3 столбца с категориальными данными.

Дропнем дубликаты, т.к. в нашем исследовании от них не будет толка.

In [5]:
df = df.drop_duplicates ()

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится, изменится лишь вектор весов линейной регрессии для новых признаков.

**Обоснование:**

Представим нашу новую матрицу признаков как результат умножения исходной матрицы признаков на некую обратимую матрицу:

$$
X_1 = XP
$$

Тогда наша формула обучения примет вид: $w_1 = (X_1^T X_1)^{-1} X_1^T y$

Подставив в нее значения $X_1$, получим: $w_1 = ((XP)^T XP)^{-1} (XP)^T y$

Воспользуемся следующими свойствами матриц, а именно:
$$A(BC) = (AB)C$$
$$(AB)^{-1} = A^{-1}B^{-1}$$
$$(AB)^T = A^TB^T$$
$$(A^T)^{-1} = (A^{-1})^T$$
$$(ABC)^{-1} =C^{-1} B^{-1} A^{-1}$$
$$AA^{-1} = E$$
$$AE=EA=A$$
Тогда нашу формулу для расчета вектора весов можно переписать:
$$
w_1 = ((XP)^T XP)^{-1} (XP)^T y \Leftrightarrow
$$
$$
w_1 = (P^T (X^T X) P)^{-1} (XP)^T y \Leftrightarrow
$$
$$
w_1 = (P^T (X^T X) P)^{-1} P^T X^T y \Leftrightarrow
$$
$$
w_1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y \Leftrightarrow
$$
$$
w_1 = P^{-1} (X^T X)^{-1} E X^T y \Leftrightarrow
$$
$$
w_1 = P^{-1}w
$$

Тоже самое, для формулы предсказаний: $a_1 = X_1w_1$, подставив получившиеся значения $X_1 = XP$ и $w_1 = P^{-1}w$ , получим:
$$a_1 = XPP^{-1}w \Leftrightarrow a_1 = Xw$$

Тем самым получаем, что наши предсказания никоем образом не изменятся.

## Алгоритм преобразования

**Алгоритм**

Нам необходимо будет:

сгенерировать обратимую матрицу, соответственно квадратную, размерностью соответствующую количеству столбцов признаков, а это 4х4.

умножить матрицу исходных признаков на сгенерированную матрицу.

посчитать качество модели на полученной матрице и сравнить с качеством модели на матрице исходных данных.

**Обоснование**

В предыдущем пункте мы выяснили, что предсказания на измененной и изначальной матрице не должны отличаться, поэтому результаты метрики R2 должны оказаться равны. Тем самым мы докажем, что правильно защитили наши данные.

## Проверка алгоритма

Создадим наши исходные признаки

In [6]:
features = df.drop('Страховые выплаты', axis=1)
target = df['Страховые выплаты']

Сгенерируем матрицу размером 4х4 методом random.normal (), т.к. при таком подходе вероятность получить необратимую матрицу близка к нулю

In [7]:
matrix_generated = np.random.normal(size=(4, 4))
display (matrix_generated)

array([[ 1.10962018, -0.05328743,  0.29728089, -0.15563279],
       [-0.37643979,  1.25931805,  0.05680509, -0.20574681],
       [ 0.17717339,  0.42621225, -0.00318726, -1.81888648],
       [-0.83271192,  1.24767591,  0.2881783 ,  0.55896089]])

Сделаем из нашей сгенерированной матрицы обратную методом linalg.inv(), т.к. в процессе обращения мы одновременно проверим матрицу на обратимость, если появится ошибка, то изначально наша сгенерированная матрица не являлась обратимой.

In [8]:
matrix_invert = np.linalg.inv(matrix_generated)
display (matrix_invert)

array([[ 0.66257407,  1.07384952, -0.45480585, -0.90020764],
       [ 0.17087668,  1.34453154, -0.30335861, -0.44466042],
       [ 0.97520643, -3.54425428,  1.29379638,  3.17700914],
       [ 0.10287173,  0.42587036, -0.6674404 , -0.19744966]])

Создадим матрицу из наших изначальных признаков, перемножим ее на сгенерированную и обращенную матрицу и составим из полученной новый набор измененных признаков

In [9]:
matrix = features.values
matrix_new = matrix@matrix_invert
features_incode = pd.DataFrame (matrix_new, columns = features.columns)

In [10]:
features_incode

,Пол,Возраст,Зарплата,Члены семьи
0,48378.010407,-175738.386926,64158.740317,157560.324454
1,37065.807607,-134619.388434,49149.640364,120705.695372
2,20484.290492,-74390.348529,27160.926503,66704.296722
3,40669.902360,-147766.316700,53943.603481,132471.548239
4,25458.334991,-92466.316055,33759.136575,82906.587773
...,...,...,...,...
4842,34819.859906,-126491.379281,46178.701713,113406.380795
4843,51106.729705,-185672.784489,67783.948487,166459.962868
4844,33063.121315,-120122.477823,43852.295105,107691.321632
4845,31893.980796,-115865.183901,42298.010490,103876.923690


Создадим класс LinearRegression для последующего обучения наших моделей

In [11]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T@X)@X.T@y
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

Обучим модель на изначальных признаках и выведем на экран метрику качества R2

In [12]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4302010044852068


Обучим модель на закодированных признаках и выведем на экран метрику качества R2

In [13]:
model = LinearRegression()
model.fit(features_incode, target)
predictions = model.predict(features_incode)
print(r2_score(target, predictions))

0.43020100448138965


Из полученного результата видим, что метрики качества на изначальных и закодированных данных отличаются незначительно, что может быть связано с округлением

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования